In [45]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")

In [17]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-714657f9-a907-4aba-9fa7-b2f1a2659057-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [18]:
# OutputParsers

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser

chain.invoke(messages)

'Ciao!'

In [21]:
# Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "italian", "text": "hi"})

# result
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

In [22]:
# Chaining together components with LCEL
chain = prompt_template | model | parser

chain.invoke({"language": "italian", "text": "hi!"})


'Ciao!'

In [41]:
# Message persistence

from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [47]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Que hago si tengo dolor de cabeza?."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Si tienes dolor de cabeza, te recomendaría descansar en un lugar tranquilo y oscuro, beber suficiente agua, y si es necesario, tomar algún analgésico como el paracetamol siguiendo las indicaciones del envase. También puedes intentar aplicar compresas frías en la frente o cuello para aliviar el dolor. Si el dolor persiste o empeora, es recomendable consultar a un médico.


In [43]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob.


In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langgraph.graph import StateGraph
from typing import Sequence
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from langgraph.checkpoint.memory import MemorySaver
from dotenv import load_dotenv

# 1. Cargar la página web
loader = WebBaseLoader("https://www.promtior.ai/")
documents = loader.load()

# 2. Dividir el texto
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(documents)

# 3. Crear embeddings y vectorstore
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(splits, embeddings)

# 4. Crear el chain de conversación
model = ChatOpenAI(model="gpt-3.5-turbo")
chain = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

# 5. Configurar LangGraph para memoria
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    context: str

workflow = StateGraph(State)

def call_model(state: State):
    # Obtener la última pregunta
    question = state["messages"][-1].content
    # Obtener historial previo
    chat_history = [(msg.content, "") for msg in state["messages"][:-1]]
    
    # Consultar usando el chain que incluye la búsqueda en la web
    response = chain({
        "question": question,
        "chat_history": chat_history
    })
    
    return {"messages": [HumanMessage(content=response["answer"])]}

# Configurar el grafo
workflow.add_node("model", call_model)
workflow.set_entry_point("model")
workflow.set_finish_point("model")

# Compilar con memoria
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# 6. Función para chatear
def chat_with_website(query, config={"configurable": {"thread_id": "promtior_chat"}}):
    input_messages = [HumanMessage(query)]
    output = app.invoke(
        {"messages": input_messages},
        config
    )
    return output["messages"][-1].content

In [3]:
# Ejemplo de uso
respuesta = chat_with_website("¿Qué servicios ofrece Promtior?")
print(respuesta)

C:\Users\csosa\AppData\Local\Temp\ipykernel_27820\82455575.py:51: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain({


Promtior ofrece una variedad de servicios, incluyendo la aceleración de la adopción de GenAI, soluciones personalizadas de GenAI desde el descubrimiento y desarrollo hasta la implementación, consultoría en la adopción de GenAI, y un departamento de GenAI como servicio que impulsa el rendimiento del equipo con talentos expertos en GenAI. Además, Promtior también ofrece soluciones completas de GenAI que incluyen análisis predictivo, automatización inteligente, entre otros.


In [57]:
# Pregunta de seguimiento
respuesta = chat_with_website("¿Puedes darme más detalles sobre esos servicios?")
print(respuesta)

Promtior ofrece servicios en diversas áreas, incluyendo servicios en el departamento de GenAI, consultoría de adopción de GenAI, y la transformación del negocio con Generative AI. En el departamento de GenAI, ofrecen servicios como la optimización de procesos de flujo de trabajo, la reducción de errores manuales, la generación automática y revisión de código, y la automatización de pruebas de software, entre otros. En el área de consultoría de adopción de GenAI, se enfocan en transformar datos complejos en ideas accionables, conectando la inteligencia artificial con los objetivos comerciales. Y en la transformación del negocio con Generative AI, ofrecen servicios como la automatización de tareas repetitivas, la personalización de comunicaciones con clientes, y la predicción de las necesidades de los clientes, entre otros.


In [58]:
# Pregunta de seguimiento
respuesta = chat_with_website("Cuando nació Trump?")
print(respuesta)

No sé la respuesta a esa pregunta.


In [59]:
# Pregunta de seguimiento
respuesta = chat_with_website("Solo sabes informacion de la pagina de Promtior? O sabes algo mas?")
print(respuesta)

No tengo información adicional además de la proporcionada sobre Promtior en este contexto.
